In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import joblib


In [10]:
# 读取G2提供的数据集
df = pd.read_csv("train_final_features.csv")
print(df.head())
print (df.shape)

encoders_g3 = {}

cat_cols = [
    "agent1_selection", "agent2_selection",
    "agent1_playout", "agent2_playout",
    "agent1_exploration", "agent2_exploration"
]

# Label encode
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    encoders_g3[col] = le
    
# 定义特征和目标列
target_col = "utility_agent1"

drop_cols = [
    target_col
]
features = [c for c in df.columns if c not in drop_cols]

X = df[features]
y = df[target_col]

print("\n--- 正在保存 G3 编码器 ---")
joblib.dump(encoders_g3, 'g3_encoders.pkl')
print("✅ G3 编码器已保存为 g3_encoders.pkl。")

   Asymmetric  NumPlayableSitesOnBoard  NumRows  NumCorners  NumDirections  \
0           0                       48        8         6.0            5.0   
1           0                       48        8         6.0            5.0   
2           0                       48        8         6.0            5.0   
3           0                       48        8         6.0            5.0   
4           0                       48        8         6.0            5.0   

   NumOrthogonalDirections  NumOuterSites  NumEdges  NumTopSites  \
0                      5.0           21.0       120            4   
1                      5.0           21.0       120            4   
2                      5.0           21.0       120            4   
3                      5.0           21.0       120            4   
4                      5.0           21.0       120            4   

   NumRightSites  ...  int_DurationTurnsNotTimeouts_x_a1_select  \
0              1  ...                                  

In [3]:
# 每一对 (agent1, agent2) 表示一场比赛
df["pair_id"] = (
    df["agent1_selection"].astype(str)
    + "_" +
    df["agent2_selection"].astype(str)
)
groups = df["pair_id"]

# GroupKFold 
gkf = GroupKFold(n_splits=5)
rmse_scores = []

In [4]:
# 训练和验证
for fold, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups)):
    print(f"\n===== Fold {fold + 1} =====")
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # 构建 LightGBM 数据集
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val)

    params = {
        "objective": "regression",
        "metric": "rmse",
        "learning_rate": 0.05,
        "num_leaves": 64,
        "feature_fraction": 0.8,
        "bagging_fraction": 0.8,
        "bagging_freq": 5,
        "verbose": -1,
        "seed": 42
    }

    model = lgb.train(
    params,
    train_data,
    valid_sets=[train_data, val_data],
    valid_names=["train", "valid"],
    num_boost_round=500,
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=100)
    ]
)

    # 预测验证集
    y_pred = model.predict(X_val, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rmse_scores.append(rmse)

    print(f"Fold {fold + 1} RMSE: {rmse:.5f}")


===== Fold 1 =====
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 0.338418	valid's rmse: 0.368298
[200]	train's rmse: 0.292007	valid's rmse: 0.331183
[300]	train's rmse: 0.269912	valid's rmse: 0.318528
[400]	train's rmse: 0.25628	valid's rmse: 0.310727
[500]	train's rmse: 0.244475	valid's rmse: 0.304603
Did not meet early stopping. Best iteration is:
[500]	train's rmse: 0.244475	valid's rmse: 0.304603
Fold 1 RMSE: 0.30460

===== Fold 2 =====
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 0.342861	valid's rmse: 0.364957
[200]	train's rmse: 0.294746	valid's rmse: 0.327831
[300]	train's rmse: 0.272976	valid's rmse: 0.311767
[400]	train's rmse: 0.259112	valid's rmse: 0.302957
[500]	train's rmse: 0.247385	valid's rmse: 0.295737
Did not meet early stopping. Best iteration is:
[500]	train's rmse: 0.247385	valid's rmse: 0.295737
Fold 2 RMSE: 0.29574

===== Fold 3 =====
Training until validation scores don't improve for 50 rou

In [5]:
# 总结
print("\n============================")
print(f"Mean RMSE across folds: {np.mean(rmse_scores):.5f}")
print("============================")

# 保存
model.save_model("lightgbm_groupkfold_baseline.txt")


Mean RMSE across folds: 0.30618


In [7]:
test_df = pd.read_csv("test.csv")
print(test_df.head())

       Id GameRulesetName                                 agent1  \
0  233234             00Y                MCTS-UCB1-0.6-NST-false   
1  233235             00Y  MCTS-ProgressiveHistory-0.1-MAST-true   
2  233236             00Y      MCTS-UCB1Tuned-0.1-Random200-true   

                                  agent2  Properties  Format  Time  Discrete  \
0  MCTS-ProgressiveHistory-0.1-MAST-true           1       1     1         1   
1                MCTS-UCB1-0.6-NST-false           1       1     1         1   
2  MCTS-ProgressiveHistory-0.1-MAST-true           1       1     1         1   

   Realtime  Turns  ...  Efficiency  CopyContext  Then  ForEachPiece  \
0         0      1  ...           1            0     1             0   
1         0      1  ...           1            0     1             0   
2         0      1  ...           1            0     1             0   

   DoLudeme  Trigger  PlayoutsPerSecond  MovesPerSecond  \
0         0        1             298.07        18877.17   